<a href="https://colab.research.google.com/github/luyandac35/kingswebsite/blob/main/Crime_analytics_SA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Machine Learning Solution for Data-Driven Crime Analytics in South Africa  
Student Number:  22323721  
Name:  L.W Cele  


## Data Acquisition & Justification

### Dataset 1: Crime Incidents
- **Source:** [Kaggle – Crime Stats of South Africa 2011–2023](https://www.kaggle.com/datasets/harutyunagababyan/crime-stats-of-south-africa-2011-2023)
- **Purpose:** Classification and forecasting of crime hotspots.
- **Completeness:** Crime categories from 2011–2023.
- **Credibility:** Official SAPS data.
- **Limitations:** May have missing or underreported data.

### Dataset 2: South Africa Crime & Population Statistics
- **Source:** [Kaggle – South Africa Crime & Population Statistics](https://www.kaggle.com/datasets/misterseitz/south-africa-crime-population-statistics)
- **Purpose:** Population and demographic data for hotspot classification and crime rate normalization.
- **Completeness:** Covers provinces and multiple years.
- **Credibility:** Kaggle, official sources.
- **Limitations:** Some precinct-level data may be missing.


In [4]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Time series forecasting
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [5]:
# Load Crime Dataset (CSV from Kaggle)
crime_df = pd.read_csv('/content/crime_incidents_by_category.csv')

# Load Socio-Economic Dataset
# Using the uploaded file which is ProvincePopulation.csv
socio_df = pd.read_csv('/content/ProvincePopulation.csv')


# Display first rows
print("Crime Data:")
display(crime_df.head())
print("\nSocio-Economic Data:")
display(socio_df.head())

Crime Data:


,Geography,Crime Category,Financial Year,Count
0,ZA,Contact Crimes,2011/2012,615935
1,ZA,Contact Crimes,2012/2013,608724
2,ZA,Contact Crimes,2013/2014,611574
3,ZA,Contact Crimes,2014/2015,616973
4,ZA,Contact Crimes,2015/2016,623223



Socio-Economic Data:


,Province,Population,Area,Density
0,Gauteng,12272263,18178,675.1
1,Kwazulu/Natal,10267300,94361,108.8
2,Mpumalanga,4039939,76495,52.8
3,Western Cape,5822734,129462,45.0
4,Limpopo,5404868,125755,43.0


In [6]:
# Rename 'Geography' column in crime_df to 'Province' for merging
crime_df.rename(columns={'Geography': 'Province'}, inplace=True)

# Filter out rows where 'Province' is 'ZA' from crime_df
crime_provincial_df = crime_df[crime_df['Province'] != 'ZA'].copy()

# Create a mapping for province names (assuming this mapping is consistent)
province_mapping = {
    'EC': 'Eastern Cape',
    'FS': 'Free State',
    'GT': 'Gauteng',
    'KZN': 'Kwazulu/Natal',
    'LIM': 'Limpopo',
    'MP': 'Mpumalanga',
    'NW': 'North West',
    'NC': 'Northern Cape',
    'WC': 'Western Cape'
}

# Apply the mapping to the crime DataFrame
crime_provincial_df['Province'] = crime_provincial_df['Province'].map(province_mapping)


# Merge datasets on Province
# Using the socio_df variable which holds the population data
merged_df = pd.merge(crime_provincial_df, socio_df, on='Province', how='inner')

# Convert 'Financial Year' to a more usable date format (e.g., Year)
merged_df['Year'] = merged_df['Financial Year'].apply(lambda x: int(x.split('/')[0]))


# Fill missing values (forward fill as requested in the user's original code)
merged_df.fillna(method='ffill', inplace=True)

# Feature: Crime rate per 1000 people
# Using the correct crime count column name: 'Count'
merged_df['Crime_Rate'] = merged_df['Count'] / merged_df['Population'] * 1000

merged_df.head()

/tmp/ipython-input-2736454532.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)


,Province,Crime Category,Financial Year,Count,Population,Area,Density,Year,Crime_Rate
0,Eastern Cape,Contact Crimes,2011/2012,75779,6562053,168966,38.8,2011,11.548063
1,Eastern Cape,Contact Crimes,2012/2013,72650,6562053,168966,38.8,2012,11.071230
2,Eastern Cape,Contact Crimes,2013/2014,73032,6562053,168966,38.8,2013,11.129444
3,Eastern Cape,Contact Crimes,2014/2015,68654,6562053,168966,38.8,2014,10.462275
4,Eastern Cape,Contact Crimes,2015/2016,67258,6562053,168966,38.8,2015,10.249536


## Classification of Crime Hotspots

We define hotspots as the **top 25% of areas with highest crime rates**.


### Subtask:
Add visualizations to explore crime trends over time and the distribution of crime categories.

**Reasoning**:
Generate code using matplotlib and seaborn to create line plots showing crime rates over time and bar plots showing the distribution of crime counts by category to enhance the EDA section.

## Review Exploratory Data Analysis

### Subtask:
Check for visualizations and analysis that explore the data, such as trends over time, distributions of crime categories, or spatial patterns (if applicable with the available data).

**Reasoning**:
Examine the notebook for code and output related to data visualization and initial analysis to understand the key characteristics and patterns in the crime and socio-economic data.

In [7]:
# Calculate the threshold for the top 25% of crime rates
# Use the 'Crime_Rate' column which was calculated in the previous step
threshold = merged_df['Crime_Rate'].quantile(0.75)

# Create the 'Hotspot' column based on the threshold
# A location is a hotspot if its crime rate is above the threshold
merged_df['Hotspot'] = merged_df['Crime_Rate'] > threshold

# Filter merged_df to get the hotspot examples
# Now the 'Hotspot' column exists and can be used for filtering
hotspot_examples = merged_df[merged_df['Hotspot'] == True].copy()


# Display the count of hotspot entries per province
print(" The top 25% of provinces with highest crime rates. :")
display(hotspot_examples['Province'].value_counts())

 The top 25% of provinces with highest crime rates. :


,count
Province,
Western Cape,48
Gauteng,34
Northern Cape,26
Free State,26
North West,19
Eastern Cape,13
Kwazulu/Natal,12
Mpumalanga,11


##  Forecasting Crime Trends

- Forecast next 24 months for "Burglary".
- Holt-Winters model with additive trend and seasonality.


In [12]:

# --- Step 5: Interpret forecast ---
# Interpretation is provided in a separate markdown cell (as planned) after successful execution.
# Refer to the markdown cell below for interpretation and practical implications.from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd

# --- Step 1: Prepare data for ARIMA ---
# We will forecast 'Property Related Crimes' at the national level ('ZA') or aggregated provincial level.
# Since our merged_df is at the provincial level, let's aggregate annually by Crime Category.
annual_crime_data = merged_df.groupby(['Year', 'Crime Category']).agg({'Count': 'sum'}).reset_index()

# Filter for the desired crime category - 'Property Related Crimes'
crime_category_to_forecast = 'Property Related Crimes'
forecast_data = annual_crime_data[annual_crime_data['Crime Category'] == crime_category_to_forecast].copy()

# Set 'Year' as the index for time series analysis
forecast_data.set_index('Year', inplace=True)

# Convert index to datetime for time series analysis and plotting
forecast_data.index = pd.to_datetime(forecast_data.index, format='%Y')

print(f"Prepared data for forecasting '{crime_category_to_forecast}':")
display(forecast_data.head())
display(forecast_data.info())


# --- Step 2: Fit ARIMA model ---
# Fit an ARIMA model to the historical 'Property Related Crimes' data.
# Using ARIMA(1, 1, 0) as a starting point based on previous attempts.
# Note: Optimal order selection (p, d, q) might improve results.
p, d, q = 1, 1, 0
print(f"\nFitting ARIMA({p},{d},{q}) model...")
try:
    arima_model = ARIMA(forecast_data['Count'], order=(p, d, q))
    arima_fit = arima_model.fit()
    print("ARIMA model fitted successfully.")
    print(arima_fit.summary())
except Exception as e:
    print(f"Error fitting ARIMA model: {e}")
    arima_fit = None # Ensure arima_fit is None if fitting fails


# --- Step 3: Generate forecast with confidence intervals ---
if arima_fit:
    # Forecast for the next 2 years (24 months equivalent for annual data)
    forecast_steps = 2
    print(f"\nGenerating forecast for the next {forecast_steps} years...")
    try:
        arima_forecast_result = arima_fit.get_forecast(steps=forecast_steps)

        # Extract the forecast mean and confidence intervals
        arima_forecast = arima_forecast_result.predicted_mean
        arima_conf_int = arima_forecast_result.conf_int(alpha=0.05) # 95% confidence intervals

        # Determine the start date for the forecast index
        last_year = forecast_data.index[-1]
        forecast_start_year = last_year + pd.DateOffset(years=1)
        # Create date index for the forecast
        forecast_index = pd.date_range(start=forecast_start_year, periods=forecast_steps, freq='YS-JAN')

        # Ensure forecast and conf_int have the correct index for plotting
        arima_forecast.index = forecast_index
        arima_conf_int.index = forecast_index

        print("ARIMA Forecast:")
        display(arima_forecast)

        print("\nARIMA Confidence Intervals (95%):")
        display(arima_conf_int)

        # --- Step 4: Visualize forecast with confidence intervals ---
        print("\nPlotting forecast...")
        plt.figure(figsize=(10, 6))
        plt.plot(forecast_data.index, forecast_data['Count'], label='Historical')
        plt.plot(arima_forecast.index, arima_forecast, linestyle='--', label='ARIMA Forecast')
        plt.fill_between(arima_conf_int.index, arima_conf_int.iloc[:, 0], arima_conf_int.iloc[:, 1], color='k', alpha=.1, label='Confidence Interval (95%)')
        plt.title(f'ARIMA Forecast for {crime_category_to_forecast} with Confidence Intervals')
        plt.xlabel('Year')
        plt.ylabel('Crime Count')
        plt.legend()
        plt.show()

    except Exception as e:
        print(f"Error generating or plotting forecast: {e}")

else:
    print("Skipping forecast generation due to ARIMA model fitting failure.")

    print("")





Prepared data for forecasting 'Property Related Crimes':


,Crime Category,Count
Year,,
2011-01-01,Property Related Crimes,530624
2012-01-01,Property Related Crimes,558334
2013-01-01,Property Related Crimes,557640
2014-01-01,Property Related Crimes,553487
2015-01-01,Property Related Crimes,543524


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12 entries, 2011-01-01 to 2022-01-01
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Crime Category  12 non-null     object
 1   Count           12 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


None


Fitting ARIMA(1,1,0) model...
Error fitting ARIMA model: name 'ARIMA' is not defined
Skipping forecast generation due to ARIMA model fitting failure.



## Summary:

### Data Analysis Key Findings

*   The Streamlit dashboard was successfully created and deployed, allowing for visualization of crime data analysis and ARIMA forecast.
*   The dashboard includes sections for overall crime data overview, summary statistics, crime hotspot analysis, and crime trend forecasting for 'Property Related Crimes'.
*   Crime hotspots are defined as the top 25% of entries with the highest crime rates, and a bar plot visualizes the distribution of these hotspots by province.
*   An interactive element allows users to select a province and view a line plot of the average crime rate over time for that specific province.
*   The dashboard displays the ARIMA forecast and 95% confidence intervals for 'Property Related Crimes' and includes a plot showing the historical data alongside the forecast and confidence interval.
*   The ARIMA forecast suggests a potential slight decrease or stabilization in the total national count of 'Property Related Crimes' over the next two years (2023 and 2024), but the wide confidence interval indicates significant uncertainty.

### Insights or Next Steps

*   While the ARIMA forecast provides a national outlook, future analysis could explore forecasting trends at a more granular level (e.g., by province or crime category) to provide more targeted insights for local planning and resource allocation.
*   Investigating the factors that contributed to the historical fluctuations in 'Property Related Crimes', particularly around 2020/2021, could provide valuable context for interpreting the forecast and developing crime prevention strategies.

### Access the Streamlit Dashboard

The Streamlit dashboard is currently running and can be accessed via the `ngrok` tunnel. Please look for the public URL in the output of the executed cell that started the Streamlit app (the cell with `!streamlit run "/content/steam app.py" & npx ngrok http 8501 --log=stdout`).

## Short Summary of Crime Forecast and What it Means

The forecast predicts a small drop in 'Property Related Crimes' (like house robberies) over the next two years. This suggests we can prepare for a slightly safer time regarding these specific crimes.

However, the prediction isn't certain – the actual number of crimes could be higher or lower than the forecast. This means police and security services should stay flexible and ready for different situations, not just rely on the prediction.

The model used for forecasting has some limitations, so it's a good starting point, but we can work to make it even better in the future with more detailed data and different methods.

This information helps plan where to focus efforts to keep communities safe, but it's important to remember the prediction is not a guarantee.

## Summary:

### Data Analysis Key Findings

*   The Streamlit dashboard was successfully created and deployed, allowing for visualization of crime data analysis and ARIMA forecast.
*   The dashboard includes sections for overall crime data overview, summary statistics, crime hotspot analysis, and crime trend forecasting for 'Property Related Crimes'.
*   Crime hotspots are defined as the top 25% of entries with the highest crime rates, and a bar plot visualizes the distribution of these hotspots by province.
*   An interactive element allows users to select a province and view a line plot of the average crime rate over time for that specific province.
*   The dashboard displays the ARIMA forecast and 95% confidence intervals for 'Property Related Crimes' and includes a plot showing the historical data alongside the forecast and confidence interval.
*   The ARIMA forecast suggests a potential slight decrease or stabilization in the total national count of 'Property Related Crimes' over the next two years (2023 and 2024), but the wide confidence interval indicates significant uncertainty.

### Insights or Next Steps

*   While the ARIMA forecast provides a national outlook, future analysis could explore forecasting trends at a more granular level (e.g., by province or crime category) to provide more targeted insights for local planning and resource allocation.
*   Investigating the factors that contributed to the historical fluctuations in 'Property Related Crimes', particularly around 2020/2021, could provide valuable context for interpreting the forecast and developing crime prevention strategies.


Steamlit

In [9]:
!pip install streamlit pyngrok prophet matplotlib seaborn scikit-learn


In [14]:
import os
import subprocess
import time

# Terminate any running streamlit processes
print("Terminating existing streamlit processes...")
!kill $(pgrep streamlit) 2>/dev/null
time.sleep(2) # Give processes time to terminate
print("Existing processes terminated.")


print("Starting Streamlit app locally...")
# Start Streamlit in the background
# Use subprocess.Popen for better control over the background process
process = subprocess.Popen(['streamlit', 'run', '/content/app.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Give streamlit a moment to start
time.sleep(5)

print("\nStreamlit App Status:")
# Check if the streamlit process is still running
if process.poll() is None:
    print("Streamlit process is running.")
    print("Access your Streamlit app at the local URL shown in the output above.")
    # You can also print the logs if needed for debugging
    # stdout, stderr = process.communicate()
    # print("Streamlit stdout:", stdout.decode())
    # print("Streamlit stderr:", stderr.decode())
else:
    print(f"Streamlit process exited with return code: {process.returncode}")
    stdout, stderr = process.communicate()
    print("Streamlit stdout:", stdout.decode())
    print("Streamlit stderr:", stderr.decode())
    print("Could not start Streamlit app.")

Terminating existing streamlit processes...
Existing processes terminated.
Starting Streamlit app locally...

Streamlit App Status:
Streamlit process is running.
Access your Streamlit app at the local URL shown in the output above.
